In [1]:
# import libraries (non-cryptogaphic)
import random # to generate phone numbers
import pandas as pd

# import libraries (cryptographic)
import cryptography.hazmat.primitives.asymmetric.dh as dh
from cryptography.fernet import Fernet
import hashlib
import sympy
import secrets

# Generate phone numbers and store them in phone_numbers.csv file

In [2]:
random.seed(10) # to ensure same phone numbers generated every time

In [3]:
# class to generate phone numbers for grab and gojek
class PhoneNumberGenerator:
    def __call__(self, count):
        phone_numbers = random.sample(range(80000000,100000000), count)
        return phone_numbers 

class FakePhoneNumberGenerator:
    def __call__(self, count):
        numbers = random.sample(range(10000000,80000000), count) # invalid phone numbers generated
        return numbers
            
    
# class to store numbers to csv
class PhoneNumberStorageManager:
    def __init__(self):
        self.filename = "phone_numbers_padded.csv"
    def __call__(self, gojek_phone_numbers, grab_phone_numbers):
        d = {"gojek": gojek_phone_numbers, 
            "grab": grab_phone_numbers}
        df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in d.items()])) # create dataframe
        df.to_csv(self.filename, index = False) # store values to file "phone_numbers_padded.csv"

### Edit cell below to change phone number count and set size

In [4]:
# set set-size
set_size = 100 # both parties are to exchange sets of a pre-determined size

# set phone number count
gojek_phone_number_count = 61 # inclusive of phone numbers in common with grab
grab_phone_number_count = 91 # inclusive of phone numbers in common with gojek
common_phone_number_count = 10



In [5]:
# instantiate required classes
phone_number_generator = PhoneNumberGenerator()
fake_phone_number_generator = FakePhoneNumberGenerator()
phone_number_storage_manager = PhoneNumberStorageManager()

# generate phone numbers
phone_numbers = phone_number_generator(gojek_phone_number_count+grab_phone_number_count-common_phone_number_count)
common_phone_numbers = phone_numbers[0:common_phone_number_count]
gojek_phone_numbers = phone_numbers[0:gojek_phone_number_count]
grab_phone_numbers = common_phone_numbers + phone_numbers[gojek_phone_number_count:] 

# generate fake phone numbers
gojek_fake_phone_numbers = fake_phone_number_generator(set_size-gojek_phone_number_count)
grab_fake_phone_numbers = fake_phone_number_generator(set_size-grab_phone_number_count)

# add fake phone numbers to phone numbers
gojek_padded_set = gojek_phone_numbers + gojek_fake_phone_numbers
grab_padded_set = grab_phone_numbers + grab_fake_phone_numbers

# shuffle phone number lists
random.shuffle(gojek_padded_set)
random.shuffle(grab_padded_set)

# write phone numbers (with fake ones) to csv file
phone_number_storage_manager(gojek_padded_set, grab_padded_set)

# Define classes for the necessary for the algorithm

In [6]:
# class to generate numbers required for psi
class NumberGenerator:
    
    def generate_public_parameters(self, size):
        # method to generate p, q and factors of p-1
        p = self.generate_safe_prime(size)
        print(f"p is prime: {sympy.ntheory.isprime(p)}")
        length_of_p = len(bin(p)[2:]) # should be 1024
        print(f"Length of prime modulus, p: {length_of_p}.\nNote: Should be {size}.")
        q = (p-1)//2
        print(f"q is prime: {sympy.ntheory.isprime(q)}")
        factors_pminus1 = [1, 2, q] # since q is prime, 2q only has these 3 factors excluding itself
                                    # 2q = p-1
                                    # factors of p-1 required to compute order of generators (lagrange theorem)
        return p, factors_pminus1

    def generate_safe_prime(self, size):
        # method to generate safe prime for p
        candidate = dh.generate_parameters(2, size).parameter_numbers().p # generate 1024-bit prime number
        while True:
            # the method used from the cryptography already generates a safe prime, this portion is merely to double confirm
            is_safe_prime = sympy.ntheory.isprime((candidate-1)//2) # if safe prime, (candidate-1)/2 is prime
            if (is_safe_prime):
                break
            else:
                candidate = dh.generate_parameters(2, size).parameter_numbers().p
                print(candidate)
 
        return candidate
    
    
    def generate_random_number(self, size):
        # method to generate client's secret
        return secrets.randbits(size)
            
# class to inspect values          
class NumberInspector:
    
    def check_is_primitive_generator(self, candidate, factors_divisorminus1, divisor): # note: factors should be the factors of divisor-1
        
        # apply lagrange theorem
        for possible_order in factors_divisorminus1: # check congruence for all factors (factors is exclusive of p-1 itself)
            result = pow(candidate, possible_order, divisor) # fast modular exponentiation implemented in pow(x,y,z)
                                                             # python integers have arbitrary precisions, 
                                                             # no overflow would occur if operations done in pure python
            if (result == 1):
                return False # if candidate exponent any of the factors (1,2,q) congruent to 1modp, candidate is not a primitive
                             # generator
        return True # order of candidate == p-1, therefore candidate is a primitive generator

    
class StorageManager:
    
    def store_data(self, filename, data):
        df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in data.items()])) # create dataframe
        df.to_csv(filename, index=False) # store data to file with filename


# Create psi client class

In [7]:
# client class (both grab and gojek are clients communicating directly with each other)
class Client:
    def __init__(self, name, other_party_name, private_key_size, phone_numbers, p, factors_pminus1, fernet_key):
        
        self.number_inspector = NumberInspector()
        self.number_generator = NumberGenerator()
        
        # for asymmetric encryption
        self.private_key = self.number_generator.generate_random_number(private_key_size)
        self.my_set = phone_numbers
        self.p = p # prime modulus
        self.factors_pminus1 = factors_pminus1 # to calculate order of hashed phone numbers, 
                                               # since algorithm requires them to be primitive generators
        
        # for symmetric encryption
        self.fernet_key = fernet_key
        self.f = Fernet(fernet_key)
        
        # values to track for psi algorithm
        self.my_hashed_set = None # h(x)
        self.my_self_encrypted_set = None # (h(x)^(my_secret))modp
        self.my_encrypted_set = None # (h(x)^(my_secret)(other_party_secret))modp
        self.other_party_encrypted_set = None # (h(y)^(my_secret)(other_party_secret))modp
        self.common_values = None # common phone numbers
        
        
        # create datafile for communication with another party
        
        # content to store in file
        # only need to share self_encrypted_values and other_party_encrypted_values
        # common_values to ensure both calculate the same intersection
        self.my_dict = {
            'my_self_encrypted_set': None, 
            'other_party_encrypted_set': None,
            'common_values': None
        }
        
        # filenames
        self.name = name
        self.filename = name + "_data_v1.2.csv"
        self.other_party_name = other_party_name
        self.other_party_filename = other_party_name + "_data_v1.2.csv"
        
        # create file
        self.storage_manager = StorageManager()
        self.storage_manager.store_data(self.filename, self.my_dict)

    def hash_to_primitive_root_modulo_p(self, element): 
        # method to hash phone numbers to primitive root modulo p i.e. primitive generator

        endian = "big"
        element = element.to_bytes(4, endian)
        hash_hex = hashlib.sha256(element).hexdigest() # sha3_256
        hash_int = int(hash_hex, 16)
        while True:
            # repeatedly hash until primitive root modulo p is obtained
            is_primitive_generator = self.number_inspector.check_is_primitive_generator(
                hash_int, self.factors_pminus1, self.p
            )
            if (is_primitive_generator):
                break
            else:
                hash_int = hash_int.to_bytes(32, endian)
                hash_hex = hashlib.sha256(hash_int).hexdigest()
                hash_int = int(hash_hex, 16)
                
        return hash_int
    
    def modular_exponentation(self, element):
        # compute (element^(private_key))modp
    
        return pow(element, self.private_key, self.p)
    
    def hash_set(self):
        # hash all phone numbers in my set to primitive root modulo p, one by one
        
        self.my_hashed_set = []
        
        for element in self.my_set:
            hashed_value = self.hash_to_primitive_root_modulo_p(element)
            self.my_hashed_set.append(hashed_value)
            
    def encrypt_set(self, is_other_party):
        # encrypt all elements in a given set using private_key, one by one
        
        # two scenarios to consider
        # one: encrypt set sent by the other party
        if (is_other_party):
            decrypted_other_party_set = self.receive_data("my_self_encrypted_set")
            other_party_set_int = []
            for element_string in decrypted_other_party_set:
                other_party_set_int.append(int(element_string))
            set_to_encrypt = other_party_set_int
        # two: encrypt my own set
        else:
            set_to_encrypt = self.my_hashed_set
        
        # encrypt values in given set, one by one
        encrypted_values = []
        for element in set_to_encrypt:
            encrypted_value = self.modular_exponentation(element)
            encrypted_values.append(encrypted_value)
            
        # assign the encrypted set to the correct variable
        # update csv file used for communication
        if (is_other_party):
            self.other_party_encrypted_set = encrypted_values
            self.send_data(encrypted_values, "other_party_encrypted_set")
        else:
            self.my_self_encrypted_set = encrypted_values
            self.send_data(encrypted_values, "my_self_encrypted_set")
                    
        
    def get_intersection(self):
        
        # get intersection
        
        my_encrypted_set = self.receive_data("other_party_encrypted_set") # read my encrypted set from the other party's file
        my_encrypted_set_int = []
        
        # convert read values to integer
        for element in my_encrypted_set:
            my_encrypted_set_int.append(int(element))
            
        # assign to correct variable
        self.my_encrypted_set = my_encrypted_set_int
        
        # get intersection
        encrypted_common_values = set(self.my_encrypted_set).intersection(self.other_party_encrypted_set)
        index_of_common_values = []
        
        # find the index of the elements in the intersection in my_encrypted_set
        for element in encrypted_common_values:
            index_of_common_values.append(self.my_encrypted_set.index(element))
            
        self.common_values = []
        
        # find the values in my own set corresponding to the index of the elements in the intersection
        for index in index_of_common_values:
            potential_phone_number = self.my_set[index]
            
            if ((potential_phone_number-79999999)>0): # remove any potential fake numbers that intersect
                self.common_values.append(potential_phone_number)
        
        # update csv file for communication
        self.send_data(self.common_values, "common_values")
    
    def encrypt_data(self, plaintext):
        # encrypt data with Fernet
        
        endian = "big"
        element = plaintext.to_bytes(128, endian) # 1024 bits == 128 bytes
        cipher_text = self.f.encrypt(element)
        return cipher_text
    
    def decrypt_data(self, ciphertext):
        # decrypt data encrypted by Fernet
        
        ciphertext_bytes = ciphertext.encode('utf-8')[2:-1] # convert from string back to bytes
        endian = "big"
        element_in_bytes = self.f.decrypt(ciphertext_bytes)
        plaintext = int.from_bytes(element_in_bytes, endian)
        return plaintext

    def send_data(self, data_to_send, column_name):
        # send data means writing to file. encrypt data with Fernet
        
        # encrypt data
        encrypted_data_to_send = []
        for element in data_to_send:
            encrypted_element = self.encrypt_data(element)
            encrypted_data_to_send.append(encrypted_element)
            
        # send data
        self.my_dict[column_name] = encrypted_data_to_send
        self.storage_manager.store_data(self.filename, self.my_dict)
        
        
    def receive_data(self, column_name):
        # receive data means reading from file (my file). decrypt data encrypted by Fernet
        
        # receive data
        encrypted_data = self.get_other_party_data()[column_name].to_list()
        
        # decrypt data
        decrypted_data = []
        for element in encrypted_data:
            if type(element) is float: # remove NaN
                continue
            decrypted_element = self.decrypt_data(element)
            decrypted_data.append(decrypted_element)
        print(f"number of elements in {self.other_party_name};{column_name}: {len(decrypted_data)}.")
        return decrypted_data
    
    def get_my_data(self):
        # read my file as dataframe (other party's file)
        
        return pd.read_csv(self.filename)
            
    def get_other_party_data(self):
        # read other party's file as dataframe
        
        return pd.read_csv(self.other_party_filename)
    
    


# Initialize context

### Edit cell below to change key size

In [8]:
# assign pre-determined variables for psi
key_size = 1024 # both private keys and large prime

In [9]:
# create key for symmetric key cryptography
fernet_key = Fernet.generate_key()

# create public parameters required for psi
number_generator = NumberGenerator()
p, factors_pminus1 = number_generator.generate_public_parameters(key_size)

# create clients
grab = Client("grab", "gojek", key_size, grab_padded_set, p, factors_pminus1, fernet_key)
gojek = Client("gojek", "grab", key_size, gojek_padded_set, p, factors_pminus1, fernet_key)



p is prime: True
Length of prime modulus, p: 1024.
Note: Should be 1024.
q is prime: True


C:\Users\jiaji\AppData\Local\Temp/ipykernel_3848/2734410534.py:55: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in data.items()])) # create dataframe


# Get intersection

## Step 1: Hash phone numbers

In [10]:
# clients hash their own set
grab.hash_set()
gojek.hash_set()

### Clients' status after step 1
Note: hashed set is not stored in the file as it is not meant to be shared with the other party hence, files' status after step 1 is not shown

In [11]:
d = {"gojek hashed set": gojek.my_hashed_set,
    "grab hashed set": grab.my_hashed_set}
df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in d.items()]))
df

,gojek hashed set,grab hashed set
0,6652014678762835950508254036300224198617716532...,1014864025876262450241425463322343998244080336...
1,2971259318188054936238701069597951289131550069...,6555446118383117843414628061923602991736146568...
2,8972686745761283250343305795976962558216403891...,2733798325363357006354662899213930682657246705...
3,3124077526244655693493627107066621456265907925...,1045592249817351765145498410716020713929588467...
4,5239384073106902936597081070089405716452234004...,1269677790173295089009790716971253424972418504...
...,...,...
95,7439334630204147692590336532878131084317999755...,2064846653753480132804612906272453669387169259...
96,1516234710201293820270438455976630785218809049...,7138528225789588782700302300052830296706897794...
97,1050996311389914722852387111588332931098320412...,3948222474783499474687611776797431125471701320...
98,5888437846314481919580418714275445066774752189...,7018556158959834041117086565402060628499966280...


## Step 2: encrypt hashed set with own private key

In [12]:
# clients self encrypt hashed set
grab.encrypt_set(False) # set is_other_party to false to encrypt own hashed set
gojek.encrypt_set(False)

C:\Users\jiaji\AppData\Local\Temp/ipykernel_3848/2734410534.py:55: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in data.items()])) # create dataframe


### Clients' status after step 2 (value of variables in client)

In [13]:
d = {"gojek hashed set": gojek.my_hashed_set,
    "grab hashed set": grab.my_hashed_set,
    "gojek self-encrypted set": gojek.my_self_encrypted_set,
    "grab self-encrypted set": grab.my_self_encrypted_set}
df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in d.items()]))
df

,gojek hashed set,grab hashed set,gojek self-encrypted set,grab self-encrypted set
0,6652014678762835950508254036300224198617716532...,1014864025876262450241425463322343998244080336...,9028087722007702628606116290034784508332114119...,1364161180404084547981143511377498255886491072...
1,2971259318188054936238701069597951289131550069...,6555446118383117843414628061923602991736146568...,3382741005917744303393919297908710323851575422...,2177113144573890130490013271418960280351636092...
2,8972686745761283250343305795976962558216403891...,2733798325363357006354662899213930682657246705...,9329392211289190668006806583883256025851967870...,2990308467255686279442634374205909559353309713...
3,3124077526244655693493627107066621456265907925...,1045592249817351765145498410716020713929588467...,6533769483996453753612252623906884496964299830...,1613891769046487084333150130946281042179503621...
4,5239384073106902936597081070089405716452234004...,1269677790173295089009790716971253424972418504...,4092304891672422099134939120155260383331137611...,9052294234244211453404401584800044603854049127...
...,...,...,...,...
95,7439334630204147692590336532878131084317999755...,2064846653753480132804612906272453669387169259...,1635293079482246503287494232981130675950509697...,1675363455860956407876543596835797053080905302...
96,1516234710201293820270438455976630785218809049...,7138528225789588782700302300052830296706897794...,3439122780817045295744838775031911844274189072...,1425048526233853251334151351936073217637626608...
97,1050996311389914722852387111588332931098320412...,3948222474783499474687611776797431125471701320...,6500663965897073604313992414652340835537805769...,1250188451301232030100085866440763119419831431...
98,5888437846314481919580418714275445066774752189...,7018556158959834041117086565402060628499966280...,1709275155456513123429094458448230390753636032...,6290727319988368493650260786564656529884606992...


### Files' status after step 2 (value of variables in file - clients' variables encrypted with Fernet)

In [14]:
df_gojek = gojek.get_my_data()
df_grab = grab.get_my_data()
print("gojek's file:")
df_gojek


gojek's file:


,my_self_encrypted_set,other_party_encrypted_set,common_values
0,b'gAAAAABhonb8tRHqVKiQWE_Bnrp-brXZCTsyvWjAlfop...,NaN,NaN
1,b'gAAAAABhonb8jSgKPtSa-uBWHqnVPOkuuh1S0-0FH5DO...,NaN,NaN
2,b'gAAAAABhonb8ZjfVPbdxA-R3kh1rmUIMopoDGQbqWVBm...,NaN,NaN
3,b'gAAAAABhonb8GdWp8f4wH78FetqDDIaixTUaIxJH-wZt...,NaN,NaN
4,b'gAAAAABhonb8P5Os8I7VIMzOhDsK2E0XrrFoV2BxTpis...,NaN,NaN
...,...,...,...
95,b'gAAAAABhonb8PWCWaWbmZyBH7cH53orkgjSKUfYgoBgD...,NaN,NaN
96,b'gAAAAABhonb8c-mm_-9M-kQHWkA-fsPIs26sGl9eE32f...,NaN,NaN
97,b'gAAAAABhonb8QPyEYo_wsLm8IqnNmsA8ytxY4UiQk0_X...,NaN,NaN
98,b'gAAAAABhonb8_AVY4Mat9NM80KN6xf8VWUevoU6JFOkr...,NaN,NaN


In [15]:
print("grab's file:")
df_grab

grab's file:


,my_self_encrypted_set,other_party_encrypted_set,common_values
0,b'gAAAAABhonb7SmCNp-o-CEg1zJRctroKgZASJS5HUzF_...,NaN,NaN
1,b'gAAAAABhonb7pguexAvHmB0oLKEYHeTLoPI0iD03hn_0...,NaN,NaN
2,b'gAAAAABhonb7itHMz7XkG2uyFBhOeJx_gJwX7mYERTg1...,NaN,NaN
3,b'gAAAAABhonb7mI0DqB5-PCiY-DnKIpNgXvtLeSOX1QKW...,NaN,NaN
4,b'gAAAAABhonb7lvOMOmhCZ8tN861OhnzAzv8gAjDdg7UN...,NaN,NaN
...,...,...,...
95,b'gAAAAABhonb7NRzawCWt0VIC5rnfD-9Zjyl1PnbnVCrK...,NaN,NaN
96,b'gAAAAABhonb7PuKfwoJLmJokYajPprxxhGac7qY2zUfC...,NaN,NaN
97,b'gAAAAABhonb7EGLZwTMha2NtAt6nRMUhuT6deXNd3qhu...,NaN,NaN
98,b'gAAAAABhonb7g5-5ztptMI0LxEzJI-ffDl-kCiNvEjDm...,NaN,NaN


## Step 3: encrypt other party's self-encrypted set with own private key


In [16]:
# clients encrypt other party's self encrypted set
grab.encrypt_set(True) # set is_other_party to true
gojek.encrypt_set(True)

number of elements in gojek;my_self_encrypted_set: 100.


C:\Users\jiaji\AppData\Local\Temp/ipykernel_3848/2734410534.py:55: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in data.items()])) # create dataframe


number of elements in grab;my_self_encrypted_set: 100.


### Clients' status after step 3

In [17]:
d = {"gojek hashed set": gojek.my_hashed_set,
    "grab hashed set": grab.my_hashed_set,
    "gojek self-encrypted set": gojek.my_self_encrypted_set,
    "grab self-encrypted set": grab.my_self_encrypted_set,
    "gojek encrypted set": grab.other_party_encrypted_set,
    "grab encrypted set": gojek.other_party_encrypted_set}
df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in d.items()]))
df

,gojek hashed set,grab hashed set,gojek self-encrypted set,grab self-encrypted set,gojek encrypted set,grab encrypted set
0,6652014678762835950508254036300224198617716532...,1014864025876262450241425463322343998244080336...,9028087722007702628606116290034784508332114119...,1364161180404084547981143511377498255886491072...,9420839751249199710646160974103883717280847244...,6033695540024023836017048013362629477945787022...
1,2971259318188054936238701069597951289131550069...,6555446118383117843414628061923602991736146568...,3382741005917744303393919297908710323851575422...,2177113144573890130490013271418960280351636092...,7638082658814174623344515217896398322587039144...,4738212856385411267384686217586350231446764864...
2,8972686745761283250343305795976962558216403891...,2733798325363357006354662899213930682657246705...,9329392211289190668006806583883256025851967870...,2990308467255686279442634374205909559353309713...,1713392689236975390415514741557256112422470195...,8804874342886021472527776147862869083110929779...
3,3124077526244655693493627107066621456265907925...,1045592249817351765145498410716020713929588467...,6533769483996453753612252623906884496964299830...,1613891769046487084333150130946281042179503621...,1648751367387722287730006613787943089720419347...,7060643444883441018496595567481225775739693627...
4,5239384073106902936597081070089405716452234004...,1269677790173295089009790716971253424972418504...,4092304891672422099134939120155260383331137611...,9052294234244211453404401584800044603854049127...,9252966054951461680579089313649767600872196649...,2613944224061241539806750360775168618865247471...
...,...,...,...,...,...,...
95,7439334630204147692590336532878131084317999755...,2064846653753480132804612906272453669387169259...,1635293079482246503287494232981130675950509697...,1675363455860956407876543596835797053080905302...,2800918139246553561681155963349011057757944162...,9811000603013078602682561511069013404011324758...
96,1516234710201293820270438455976630785218809049...,7138528225789588782700302300052830296706897794...,3439122780817045295744838775031911844274189072...,1425048526233853251334151351936073217637626608...,1431780043972275202741300171227558593254514924...,1328892031233222764663572861126358559033619665...
97,1050996311389914722852387111588332931098320412...,3948222474783499474687611776797431125471701320...,6500663965897073604313992414652340835537805769...,1250188451301232030100085866440763119419831431...,1121709401695176891980823261773529641684334453...,1588559476413798022873243105443461045148703077...
98,5888437846314481919580418714275445066774752189...,7018556158959834041117086565402060628499966280...,1709275155456513123429094458448230390753636032...,6290727319988368493650260786564656529884606992...,1618203856047219900307848813567057533877774884...,1630642601604876709531006063314939968524415809...


### Files' status after step 3

In [18]:
df_gojek = gojek.get_my_data()
df_grab = grab.get_my_data()
print("gojek's file:")
df_gojek



gojek's file:


,my_self_encrypted_set,other_party_encrypted_set,common_values
0,b'gAAAAABhonb8tRHqVKiQWE_Bnrp-brXZCTsyvWjAlfop...,b'gAAAAABhoncAGSics1oKeHgfKSdgB9FOIJZGjE5kYLZc...,NaN
1,b'gAAAAABhonb8jSgKPtSa-uBWHqnVPOkuuh1S0-0FH5DO...,b'gAAAAABhoncAUGjK0GqHXi1LdSPlRo4hWguEhaa4hOG7...,NaN
2,b'gAAAAABhonb8ZjfVPbdxA-R3kh1rmUIMopoDGQbqWVBm...,b'gAAAAABhoncAi18osWcOMzZD6h5uNiXBpjc713jyp8UQ...,NaN
3,b'gAAAAABhonb8GdWp8f4wH78FetqDDIaixTUaIxJH-wZt...,b'gAAAAABhoncAALiN0VZlhJELTciofBbn-YsdW-iKdMac...,NaN
4,b'gAAAAABhonb8P5Os8I7VIMzOhDsK2E0XrrFoV2BxTpis...,b'gAAAAABhoncA7QD71m92b1bYYSOFtmloeb-IK0JUftiZ...,NaN
...,...,...,...
95,b'gAAAAABhonb8PWCWaWbmZyBH7cH53orkgjSKUfYgoBgD...,b'gAAAAABhoncAkKV66tH3qNenBJJq1h7gBDJEogw_KJ5Z...,NaN
96,b'gAAAAABhonb8c-mm_-9M-kQHWkA-fsPIs26sGl9eE32f...,b'gAAAAABhoncA7GzJ_wHOts-P4nfGjD3rTQECgTCwH0jD...,NaN
97,b'gAAAAABhonb8QPyEYo_wsLm8IqnNmsA8ytxY4UiQk0_X...,b'gAAAAABhoncAY-9GhFwCt4I8DKTsOZVmctDFdyAFcL0G...,NaN
98,b'gAAAAABhonb8_AVY4Mat9NM80KN6xf8VWUevoU6JFOkr...,b'gAAAAABhoncAd1yyRp4FZWpFXz4Ilh0GzbEqTIXXPsdo...,NaN


In [19]:
print("grab's file:")
df_grab

grab's file:


,my_self_encrypted_set,other_party_encrypted_set,common_values
0,b'gAAAAABhonb7SmCNp-o-CEg1zJRctroKgZASJS5HUzF_...,b'gAAAAABhonb-PI9VHtAq0qztZf-2GHKvsFA9IVqgZkhh...,NaN
1,b'gAAAAABhonb7pguexAvHmB0oLKEYHeTLoPI0iD03hn_0...,b'gAAAAABhonb-WtbS_YSK9Jmf_AR1VgTCGb8GhDwLu32t...,NaN
2,b'gAAAAABhonb7itHMz7XkG2uyFBhOeJx_gJwX7mYERTg1...,b'gAAAAABhonb-25x09S5zU-FTQC4ufDcOm-HOJAgGvBL_...,NaN
3,b'gAAAAABhonb7mI0DqB5-PCiY-DnKIpNgXvtLeSOX1QKW...,b'gAAAAABhonb-2XnVVCyYscxZ6iqBnw7MVD4Ny3Xe-Zr4...,NaN
4,b'gAAAAABhonb7lvOMOmhCZ8tN861OhnzAzv8gAjDdg7UN...,b'gAAAAABhonb-zeyYNgCR-NXXjiS7qV1q3O_DuBr1LVXZ...,NaN
...,...,...,...
95,b'gAAAAABhonb7NRzawCWt0VIC5rnfD-9Zjyl1PnbnVCrK...,b'gAAAAABhonb-QN9q6EkGTADgPuIhUMrjzE1_xNfiRLoZ...,NaN
96,b'gAAAAABhonb7PuKfwoJLmJokYajPprxxhGac7qY2zUfC...,b'gAAAAABhonb-s17OdTob15DCIfkbeXk_tacgBBmBUrQD...,NaN
97,b'gAAAAABhonb7EGLZwTMha2NtAt6nRMUhuT6deXNd3qhu...,b'gAAAAABhonb-j5J05VXBgEvuXs6Tdu6ZE0rGbd9dR0y6...,NaN
98,b'gAAAAABhonb7g5-5ztptMI0LxEzJI-ffDl-kCiNvEjDm...,b'gAAAAABhonb-SWbSHa4v2_NPTHL3osaWHOuOIED3wKMR...,NaN


## Step 4: find intersection


In [20]:
# clients find intersection
grab.get_intersection()
gojek.get_intersection()

number of elements in gojek;other_party_encrypted_set: 100.
number of elements in grab;other_party_encrypted_set: 100.


### Clients' status after step 4:

In [21]:
d = {"gojek hashed set": gojek.my_hashed_set,
    "grab hashed set": grab.my_hashed_set,
    "gojek self-encrypted set": gojek.my_self_encrypted_set,
    "grab self-encrypted set": grab.my_self_encrypted_set,
    "gojek encrypted set": grab.other_party_encrypted_set,
    "grab encrypted set": gojek.other_party_encrypted_set,
    "gojek found intersection": gojek.common_values,
    "grab found intersection:": grab.common_values}
df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in d.items()]))
df

,gojek hashed set,grab hashed set,gojek self-encrypted set,grab self-encrypted set,gojek encrypted set,grab encrypted set,gojek found intersection,grab found intersection:
0,6652014678762835950508254036300224198617716532...,1014864025876262450241425463322343998244080336...,9028087722007702628606116290034784508332114119...,1364161180404084547981143511377498255886491072...,9420839751249199710646160974103883717280847244...,6033695540024023836017048013362629477945787022...,96485172.0,96485172.0
1,2971259318188054936238701069597951289131550069...,6555446118383117843414628061923602991736146568...,3382741005917744303393919297908710323851575422...,2177113144573890130490013271418960280351636092...,7638082658814174623344515217896398322587039144...,4738212856385411267384686217586350231446764864...,80497694.0,80497694.0
2,8972686745761283250343305795976962558216403891...,2733798325363357006354662899213930682657246705...,9329392211289190668006806583883256025851967870...,2990308467255686279442634374205909559353309713...,1713392689236975390415514741557256112422470195...,8804874342886021472527776147862869083110929779...,99397525.0,99397525.0
3,3124077526244655693493627107066621456265907925...,1045592249817351765145498410716020713929588467...,6533769483996453753612252623906884496964299830...,1613891769046487084333150130946281042179503621...,1648751367387722287730006613787943089720419347...,7060643444883441018496595567481225775739693627...,86915509.0,86915509.0
4,5239384073106902936597081070089405716452234004...,1269677790173295089009790716971253424972418504...,4092304891672422099134939120155260383331137611...,9052294234244211453404401584800044603854049127...,9252966054951461680579089313649767600872196649...,2613944224061241539806750360775168618865247471...,96192082.0,96192082.0
...,...,...,...,...,...,...,...,...
95,7439334630204147692590336532878131084317999755...,2064846653753480132804612906272453669387169259...,1635293079482246503287494232981130675950509697...,1675363455860956407876543596835797053080905302...,2800918139246553561681155963349011057757944162...,9811000603013078602682561511069013404011324758...,NaN,NaN
96,1516234710201293820270438455976630785218809049...,7138528225789588782700302300052830296706897794...,3439122780817045295744838775031911844274189072...,1425048526233853251334151351936073217637626608...,1431780043972275202741300171227558593254514924...,1328892031233222764663572861126358559033619665...,NaN,NaN
97,1050996311389914722852387111588332931098320412...,3948222474783499474687611776797431125471701320...,6500663965897073604313992414652340835537805769...,1250188451301232030100085866440763119419831431...,1121709401695176891980823261773529641684334453...,1588559476413798022873243105443461045148703077...,NaN,NaN
98,5888437846314481919580418714275445066774752189...,7018556158959834041117086565402060628499966280...,1709275155456513123429094458448230390753636032...,6290727319988368493650260786564656529884606992...,1618203856047219900307848813567057533877774884...,1630642601604876709531006063314939968524415809...,NaN,NaN


Note: Last 2 columns, unlike the rest of the columns, do not have a one-to-one mapping with other values belonging to the same row i.e. values in the last 2 columns do not have any relation to the other values in the same row as it.

### Files' status after step 4

In [22]:
df_gojek = gojek.get_my_data()
df_grab = grab.get_my_data()
print("gojek's file:")
df_gojek


gojek's file:


,my_self_encrypted_set,other_party_encrypted_set,common_values
0,b'gAAAAABhonb8tRHqVKiQWE_Bnrp-brXZCTsyvWjAlfop...,b'gAAAAABhoncAGSics1oKeHgfKSdgB9FOIJZGjE5kYLZc...,b'gAAAAABhoncAaVig_LHUTn-aLW6zi5FKSb9f6-f2dSSg...
1,b'gAAAAABhonb8jSgKPtSa-uBWHqnVPOkuuh1S0-0FH5DO...,b'gAAAAABhoncAUGjK0GqHXi1LdSPlRo4hWguEhaa4hOG7...,b'gAAAAABhoncA8vifuF8s-rjcukMspPceY0vbM_VSNY_c...
2,b'gAAAAABhonb8ZjfVPbdxA-R3kh1rmUIMopoDGQbqWVBm...,b'gAAAAABhoncAi18osWcOMzZD6h5uNiXBpjc713jyp8UQ...,b'gAAAAABhoncA-UmuuusoyiUbC0ulXV2hvjZF3Xrf3NbE...
3,b'gAAAAABhonb8GdWp8f4wH78FetqDDIaixTUaIxJH-wZt...,b'gAAAAABhoncAALiN0VZlhJELTciofBbn-YsdW-iKdMac...,b'gAAAAABhoncA85ANg8E5B6t3q5u1fnyv2vl2p1lFlph0...
4,b'gAAAAABhonb8P5Os8I7VIMzOhDsK2E0XrrFoV2BxTpis...,b'gAAAAABhoncA7QD71m92b1bYYSOFtmloeb-IK0JUftiZ...,b'gAAAAABhoncAqMTlFQS1hA3aRXnWe6aGlH_S2bD0KDPn...
...,...,...,...
95,b'gAAAAABhonb8PWCWaWbmZyBH7cH53orkgjSKUfYgoBgD...,b'gAAAAABhoncAkKV66tH3qNenBJJq1h7gBDJEogw_KJ5Z...,NaN
96,b'gAAAAABhonb8c-mm_-9M-kQHWkA-fsPIs26sGl9eE32f...,b'gAAAAABhoncA7GzJ_wHOts-P4nfGjD3rTQECgTCwH0jD...,NaN
97,b'gAAAAABhonb8QPyEYo_wsLm8IqnNmsA8ytxY4UiQk0_X...,b'gAAAAABhoncAY-9GhFwCt4I8DKTsOZVmctDFdyAFcL0G...,NaN
98,b'gAAAAABhonb8_AVY4Mat9NM80KN6xf8VWUevoU6JFOkr...,b'gAAAAABhoncAd1yyRp4FZWpFXz4Ilh0GzbEqTIXXPsdo...,NaN


In [23]:
print("grab's file:")
df_grab

grab's file:


,my_self_encrypted_set,other_party_encrypted_set,common_values
0,b'gAAAAABhonb7SmCNp-o-CEg1zJRctroKgZASJS5HUzF_...,b'gAAAAABhonb-PI9VHtAq0qztZf-2GHKvsFA9IVqgZkhh...,b'gAAAAABhoncAq_KSPoZk96MxUddp-7015-dF7i8JGUHd...
1,b'gAAAAABhonb7pguexAvHmB0oLKEYHeTLoPI0iD03hn_0...,b'gAAAAABhonb-WtbS_YSK9Jmf_AR1VgTCGb8GhDwLu32t...,b'gAAAAABhoncAO57LJAB40IY2Zy5X-RWi409yCIbJ2hJh...
2,b'gAAAAABhonb7itHMz7XkG2uyFBhOeJx_gJwX7mYERTg1...,b'gAAAAABhonb-25x09S5zU-FTQC4ufDcOm-HOJAgGvBL_...,b'gAAAAABhoncA1Etr5xSYwW9vTAAmEgjEJPhExixIUnh8...
3,b'gAAAAABhonb7mI0DqB5-PCiY-DnKIpNgXvtLeSOX1QKW...,b'gAAAAABhonb-2XnVVCyYscxZ6iqBnw7MVD4Ny3Xe-Zr4...,b'gAAAAABhoncAfxYmTvJTTKDg6_5rSQKwpET1dN2csjHY...
4,b'gAAAAABhonb7lvOMOmhCZ8tN861OhnzAzv8gAjDdg7UN...,b'gAAAAABhonb-zeyYNgCR-NXXjiS7qV1q3O_DuBr1LVXZ...,b'gAAAAABhoncAwvtY3dEztUtZqE_yIc6aDF6JaZX0-quI...
...,...,...,...
95,b'gAAAAABhonb7NRzawCWt0VIC5rnfD-9Zjyl1PnbnVCrK...,b'gAAAAABhonb-QN9q6EkGTADgPuIhUMrjzE1_xNfiRLoZ...,NaN
96,b'gAAAAABhonb7PuKfwoJLmJokYajPprxxhGac7qY2zUfC...,b'gAAAAABhonb-s17OdTob15DCIfkbeXk_tacgBBmBUrQD...,NaN
97,b'gAAAAABhonb7EGLZwTMha2NtAt6nRMUhuT6deXNd3qhu...,b'gAAAAABhonb-j5J05VXBgEvuXs6Tdu6ZE0rGbd9dR0y6...,NaN
98,b'gAAAAABhonb7g5-5ztptMI0LxEzJI-ffDl-kCiNvEjDm...,b'gAAAAABhonb-SWbSHa4v2_NPTHL3osaWHOuOIED3wKMR...,NaN


# Check results

In [24]:
# get intersection found by the two parties
gojek_found_intersection = gojek.common_values
grab_found_intersection = grab.common_values


# sort numbers for easier comparison
gojek_found_intersection.sort()
grab_found_intersection.sort()
common_phone_numbers.sort()

# summarize them in a dataframe
d = {"actual": common_phone_numbers,
    "gojek": gojek_found_intersection,
    "grab": grab_found_intersection}
df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in d.items()]))
df

,actual,gojek,grab
0,80497694,80497694,80497694
1,81093373,81093373,81093373
2,86915509,86915509,86915509
3,89312048,89312048,89312048
4,94391128,94391128,94391128
5,95521626,95521626,95521626
6,96192082,96192082,96192082
7,96485172,96485172,96485172
8,99173089,99173089,99173089
9,99397525,99397525,99397525
